# Structuring the Quantitative Data: Separating data for each scenario
### _This script only works for iteration 3 of the design_

Date: Jun 4, 2019

Last Modified: June 6, 2019
SH

** The purpose is to cut out the data collected during the time when the sceanrio was running but partipant or AV were not driving (e.g. turining the turntable, calibrating the eyetracker)

__This script needs to be run for each participant__

#### Part A. Simulator Raw Data -> Simulator Structured Data



In [724]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [725]:
# Initialization
import numpy as np
import pandas as pd
from datetime import timedelta, datetime
from os import listdir
import re
import glob

import matplotlib.pyplot as plt
plt.style.use('seaborn-darkgrid')

In [726]:
## Setting the path
Quantitative_path = "N:\\Shabnam Haghzare\\01- AV_OA_Study\\Master_Data_List\\Quantitative"

In [727]:
ID = input('Enter the ID of the participant for their data to be structured:')

Enter the ID of the participant for their data to be structured:22


In [728]:
number_of_empatica_files = int(input("indicate the number of empatica files created for the participant due to connection break?"))

indicate the number of empatica files created for the participant due to connection break?1


## Part A. Structuing Simulator Raw data based on the Time Markers 

This script will: 

1. Read simlator data from "[ID]/Raw/Simulator/...";
2. Crop both simulator generated excel sheets (simulator_outputs and SH_simulator_outputs) to include the time frame between the time-markers of "start driving" and "stop driving" in the scenarios.
3. Save the structured outputs in "[ID]/Structured/simulator/..."



In [729]:
TimeMarkers_df = pd.DataFrame()

for scenario in ['BA','BM','RA','RM','TA','TM']:
    scenario_path_raw = Quantitative_path + "\\" + ID + "\\Raw\\Simulator\\" + scenario
    TM_path = (glob.glob(scenario_path_raw + "\\*.tm"))[0]
    
    
    ## 1. Getting the start time and stop time from the TimeMarker file
    print('\n', TM_path, '\n')
    TM_text= open(TM_path, "r")
    
    start_sec_string = TM_text.readlines(400)[2]
    
    TM_text= open(TM_path, "r")
    stop_sec_string = TM_text.readlines(400)[17]
    TM_text.close()
    
    start_sec = int(''.join(x for x in start_sec_string if x.isdigit()))/1000
    print("start sec:", start_sec )
    stop_sec = int(''.join(x for x in stop_sec_string if x.isdigit()))/1000
    print("stop sec:", stop_sec )
    
    TimeMarkers_df.loc["start_sec", scenario] = start_sec
    TimeMarkers_df.loc["stop_sec", scenario] = stop_sec
    
    # 2. read two csv files for each scenario
    raw_simulator_outputs = pd.read_csv(scenario_path_raw + "\\simulator_outputs.csv") 
    raw_SH_simulator_outputs = pd.read_csv(scenario_path_raw + "\\SH_simulator_outputs.csv")  

    # 3. Crop them nased on the starting and stopping times

    structured_simulator_outputs = raw_simulator_outputs.set_index('Time').truncate(before= start_sec, after = stop_sec)
    structured_SH_simulator_outputs = raw_SH_simulator_outputs.set_index('Time').truncate(before= start_sec, after = stop_sec)
    
    
    # 4. save them in the [ID]/Structured/[scenario] folder

    scenario_path_Structured = Quantitative_path + "\\" + ID + "\\Structured\\Simulator\\" + scenario

    structured_SH_simulator_outputs.to_csv(scenario_path_Structured + "\\" + scenario + "_SH_simulator_outputs.csv")
    structured_simulator_outputs.to_csv(scenario_path_Structured + "\\" + scenario + "_simulator_outputs.csv")


 N:\Shabnam Haghzare\01- AV_OA_Study\Master_Data_List\Quantitative\22\Raw\Simulator\BA\TimeMarkers-RECORD-Experimental_Normal_Auto_AV&OA-21_05_2019-09h35m23s.tm 

start sec: 107.399
stop sec: 542.849

 N:\Shabnam Haghzare\01- AV_OA_Study\Master_Data_List\Quantitative\22\Raw\Simulator\BM\TimeMarkers-RECORD-Experimental_Normal_Manual_AV&OA-21_05_2019-09h13m07s.tm 

start sec: 611.697
stop sec: 1132.703

 N:\Shabnam Haghzare\01- AV_OA_Study\Master_Data_List\Quantitative\22\Raw\Simulator\RA\TimeMarkers-RECORD-Experimental_Rainy_Auto_AV&OA-21_05_2019-10h53m55s.tm 

start sec: 28.299
stop sec: 503.605

 N:\Shabnam Haghzare\01- AV_OA_Study\Master_Data_List\Quantitative\22\Raw\Simulator\RM\TimeMarkers-RECORD-Experimental_Rainy_Manual_AV&OA-21_05_2019-10h40m34s.tm 

start sec: 175.699
stop sec: 688.053

 N:\Shabnam Haghzare\01- AV_OA_Study\Master_Data_List\Quantitative\22\Raw\Simulator\TA\TimeMarkers-RECORD-Experimental_Traffic_Auto_AV&OA-21_05_2019-10h14m37s.tm 

start sec: 95.65
stop sec: 60

## Part B. Making the TimeMarker Data Frame to reflect the actual start and top times of scenarios

In [730]:
for scenario in ['BA','BM','RA','RM','TA','TM']:
    
    scenario_path_raw = Quantitative_path + "\\" + ID + "\\Raw\\Simulator\\" + scenario
    TM_path = (glob.glob(scenario_path_raw + "\\*.tm"))[0]
    
    scenario_run_datetime = datetime.strptime(TM_path[-23:-3], '%d_%m_%Y-%Hh%Mm%Ss')
    
    TimeMarkers_df.loc["start_datetime", scenario] = scenario_run_datetime + timedelta(seconds=start_sec)
    TimeMarkers_df.loc["stop_datetime", scenario] = scenario_run_datetime + timedelta(seconds=stop_sec)

    TimeMarkers_df.loc["start_UNIX", scenario] = (TimeMarkers_df.loc["start_datetime", scenario]).timestamp()
    TimeMarkers_df.loc["stop_UNIX", scenario] = (TimeMarkers_df.loc["stop_datetime", scenario]).timestamp()


In [731]:
TimeMarkers_df.to_csv(Quantitative_path + "\\" + ID + "\\Structured\\TimeMarkers.csv", date_format='%Y-%m-%d %H:%M:%S')
TimeMarkers_df

,BA,BM,RA,RM,TA,TM
start_sec,107.399,611.697,28.299,175.699,95.65,45.05
stop_sec,542.849,1132.7,503.605,688.053,601.501,600.097
start_datetime,2019-05-21 09:36:08.050000,2019-05-21 09:13:52.050000,2019-05-21 10:54:40.050000,2019-05-21 10:41:19.050000,2019-05-21 10:15:22.050000,2019-05-21 10:03:08.050000
stop_datetime,2019-05-21 09:45:23.097000,2019-05-21 09:23:07.097000,2019-05-21 11:03:55.097000,2019-05-21 10:50:34.097000,2019-05-21 10:24:37.097000,2019-05-21 10:12:23.097000
start_UNIX,1.55845e+09,1.55844e+09,1.55845e+09,1.55845e+09,1.55845e+09,1.55845e+09
stop_UNIX,1.55845e+09,1.55844e+09,1.55845e+09,1.55845e+09,1.55845e+09,1.55845e+09


## Part C. Structuing Empatica Raw data based on the Simulator Generated Time Markers

This script will: 
1.Read Empatica data from "[ID]/Raw/Empatica/...";

2.Crop each physiological signal collected by Empatica (BVP, HR, EDA, TEMP, IBI) to include the time frame between the time-markers of "start driving" and "stop driving" in the scenarios.

3.Save the structured outputs in "[ID]/Structured/Empatica/[scenarios]"


In [732]:
## Initialization of sampling frequencies based on Empatica Instructions
HR_fs = 1 #Hz
Temp_fs = 4 #Hz
EDA_fs = 4 #Hz

## Empatica Path
empatica_path_raw = Quantitative_path + "\\" + ID + "\\Raw\\Empatica\\"
empatica_path_structured = Quantitative_path + "\\" + ID + "\\Structured\\Empatica\\"

In [733]:
def raw_phys_to_phys_df(num, phys_string, phys_sig_fs):
    raw_phys_sig = np.genfromtxt(empatica_path_raw + phys_string + " ("+ str(num)+")" +".csv", delimiter=',', dtype = None)
    raw_phys_sig_start_datetime = pd.to_datetime(raw_phys_sig[0], unit = 's') - timedelta(hours = 4)
    raw_phys_sig = raw_phys_sig[2:]
    raw_phys_sig_num_data_points = raw_phys_sig.shape[0]

    ## Adding a Time Series for each sample
    datetime_series = pd.Series()

    for sample in range(raw_phys_sig_num_data_points):
        datetime_series.loc[sample] = raw_phys_sig_start_datetime + timedelta(seconds = sample*(1/phys_sig_fs))
    
    raw_phys_sig_df = pd.DataFrame({"Time" : datetime_series, phys_string : raw_phys_sig}) 
    
    return raw_phys_sig_df

In [734]:
def structuring_empatica_signals(phys_string , phys_sig_fs):
    raw_phys_sig = np.genfromtxt(empatica_path_raw + phys_string + ".csv", delimiter=',', dtype = None)
    raw_phys_sig_start_datetime = pd.to_datetime(raw_phys_sig[0], unit = 's') - timedelta(hours = 4)
    raw_phys_sig = raw_phys_sig[2:]
    raw_phys_sig_num_data_points = raw_phys_sig.shape[0]

    ## Adding a Time Series for each sample
    datetime_series = pd.Series()

    for sample in range(raw_phys_sig_num_data_points):
        datetime_series.loc[sample] = raw_phys_sig_start_datetime + timedelta(seconds = sample*(1/phys_sig_fs))
    
    raw_phys_sig_df = pd.DataFrame({"Time" : datetime_series, phys_string : raw_phys_sig}) 


    if number_of_empatica_files != 1:
        for n in range(number_of_empatica_files-1):
            num = n + 2
            raw_phys_sig_df = pd.concat([raw_phys_sig_df, raw_phys_to_phys_df(num, phys_string, phys_sig_fs)])

    for scenario in ['BA','BM','RA','RM','TA','TM']:
        df_copy = raw_phys_sig_df.copy()
        t_start = pd.Timestamp(TimeMarkers_df.loc["start_datetime", scenario])
        t_stop  = pd.Timestamp(TimeMarkers_df.loc["stop_datetime", scenario])
    
        structured_physio_outputs = (df_copy.set_index('Time')).truncate(before= t_start, after = t_stop)
        structured_physio_outputs.to_csv(empatica_path_structured + scenario + "\\" + scenario + "_" + phys_string+".csv", date_format='%Y-%m-%d %H:%M:%S%f')
    return structured_physio_outputs

#### Heart rate

In [735]:
## HR Reading and structuring it based on scenario and saving it
structuring_empatica_signals(phys_string = "HR" , phys_sig_fs = HR_fs)

,HR
Time,
2019-05-21 10:03:09,61.33
2019-05-21 10:03:10,61.85
2019-05-21 10:03:11,62.38
2019-05-21 10:03:12,62.90
2019-05-21 10:03:13,63.27
2019-05-21 10:03:14,63.67
2019-05-21 10:03:15,64.03
2019-05-21 10:03:16,64.42
2019-05-21 10:03:17,64.65


#### Electro-dermal Activity (EDA)

In [736]:
## EDA Reading and structuring it based on scenario and saving it
structuring_empatica_signals(phys_string = "EDA" , phys_sig_fs = EDA_fs)

,EDA
Time,
2019-05-21 10:03:08.250,0.055091
2019-05-21 10:03:08.500,0.053809
2019-05-21 10:03:08.750,0.051247
2019-05-21 10:03:09.000,0.060215
2019-05-21 10:03:09.250,0.058934
2019-05-21 10:03:09.500,0.058934
2019-05-21 10:03:09.750,0.058934
2019-05-21 10:03:10.000,0.060215
2019-05-21 10:03:10.250,0.058934


#### Temperature

In [737]:
## Temp Reading and structuring it based on scenario and saving it
structuring_empatica_signals(phys_string = "TEMP" , phys_sig_fs = Temp_fs)

,TEMP
Time,
2019-05-21 10:03:08.250,29.71
2019-05-21 10:03:08.500,29.71
2019-05-21 10:03:08.750,29.71
2019-05-21 10:03:09.000,29.71
2019-05-21 10:03:09.250,29.71
2019-05-21 10:03:09.500,29.71
2019-05-21 10:03:09.750,29.71
2019-05-21 10:03:10.000,29.73
2019-05-21 10:03:10.250,29.73


## End of Script